# INTRODUCCIÓN AL NLP (NATURAL LANGUAGE PROCESSING - PROCESAMIENTO DE LENGUAJE NATURAL)

## Conceptos básicos.

### TOKENIZACIÓN

En un proceso de NLP una de las primeras tareas a realizar va a ser eliminar las palabras que tienen poco interés para nuestro análisis.   
El primer paso es delimitar las palabras del texto, y convertir esas palabras en elementos de una lista. Este procedimiento es conocido como **tokenización**. 

Procedemos a implementar esto en Python: (Usaremos la librería ***SPACY***-> **pip install spacy**)

Nota:    
Ejecutar en terminal: 
     
***python -m spacy download es***   

para descargar el modelo de idioma español-castellano

In [6]:
import spacy
nlp = spacy.load('es_core_news_sm')
text = """Soy un texto. Normalmente soy más largo y más grande. Que no te engañe mi tamaño."""
doc = nlp(text) # Crea un objeto de spacy tipo nlp
tokens = [t.orth_ for t in doc] # Crea una lista con las palabras del texto

En este punto, ya tenemos nuestro texto convertido en una lista de **tokens**.    

Lamentablemente, están todas las palabras. Se puede comprobar que también se incluyen como **tokens** los signos de puntuación.    

Por nuestra parte, lo que nos interesa es quedarnos solo con aquellas que sean más o menos representativas del texto. Para ello, vamos a eliminar de esa lista las palabras muy comunes o poco informativas.

A continuación, vamos a pedirle a esta librería que construya la lista de tokens pero que no incluya palabras muy comunes y poco informativas desde el punto de vista léxico, tales como conjunciones (y, o, ni, que), preposiciones (a, en, para, por, entre otras) y verbos muy comunes (ser, ir, y otros más).    

Para realizarlo, se va a utilizar una condición que diga “todos los tokens del texto, siempre y cuando no se incluyan la puntuación ni las palabras poco representativas (stopwords)”.

In [7]:
import spacy
nlp = spacy.load('es_core_news_sm')
text = """Soy un texto. Normalmente soy más largo y más grande. Que no te engañe mi tamaño."""
doc = nlp(text)
lexical_tokens = [t.orth_ for t in doc if not t.is_punct | t.is_stop]

Ahora ya disponemos de una lista de palabra que nos pueden dar una idea sobre la temática tratada en el texto o bien, que nos permita una clasificación más certera.

## NORMALIZACIÓN

El siguiente paso en este flujo de trabajo consiste en normalizar el texto.       

El tokenizador reconoce formas como *caminar, Caminar y CAMINAR* como formas distintas. Además, el documento puede tener números y palabras compuestas por caracteres alfanuméricos y otros símbolos tales como #Ar1anaG. Si no nos interesan estas palabras, y queremos que en la lista aparezcan solamente las formas convencionales (por ejemplo, caminar, sólo en minúsculas) debemos normalizar nuestro texto.    

Aprovecharemos el **momentum** para descartar palabras muy cortas (menores a 4 caracteres) para filtrar aún más nuestros tokens.


In [8]:
words = [t.lower() for t in lexical_tokens if len(t) > 3 and t.isalpha()]

Si lo agrupamos todo en una única función:

In [10]:
import spacy
nlp = spacy.load('es_core_news_sm')
def normalize(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and     
    t.isalpha()]
    return lexical_tokens

word_list = normalize("Soy un texto de prueba. ¿Cuántos tokens me quedarán después de la normalización?")
print(word_list)

['texto', 'prueba', 'tokens', 'quedarán', 'normalización']


## LEMATIZACIÓN

En español, por ejemplo, sabemos que canto, cantas, canta, cantamos, cantáis, cantan son distintas formas (conjugaciones) de un mismo verbo (cantar). Y que niña, niño, niñita, niños, niñotes, y otras más, son distintas formas del vocablo niño. Así que sería genial poder obviar las diferencias y juntar todas estas variantes en un mismo término.    
Y eso es precisamente lo que hace la lematización: relaciona una palabra flexionada o derivada con su forma canónica o lema. Y un lema no es otra cosa que la forma que tienen las palabras cuando las buscas en el diccionario. 

In [11]:
import spacy
nlp = spacy.load('es_core_news_sm')
text = """Soy un texto que pide a gritos que lo procesen. Por eso yo canto, tú cantas, ella canta, nosotros cantamos, cantáis, cantan…"""
doc = nlp(text)
lemmas = [tok.lemma_.lower() for tok in doc]
print(lemmas)

['ser', 'uno', 'texto', 'que', 'pedir', 'a', 'grito', 'que', 'él', 'procesen', '.', 'por', 'ese', 'yo', 'cantar', ',', 'tú', 'canta', ',', 'él', 'cantar', ',', 'yo', 'cantar', ',', 'cantáis', ',', 'cantar', '…']


Como el proceso de lematización toma en consideración la probable clase de palabra (adjetivo, verbo, sustantivo…) — también llamados POS — es posible usar dicha información para filtrar la lista de lemas.   
La siguiente línea excluye los pronombres de nuestra lista de lemas:

In [12]:
lemmas_no_pron = [tok.lemma_.lower() for tok in doc if tok.pos_ != 'PRON']
print(lemmas_no_pron)

['ser', 'uno', 'texto', 'pedir', 'a', 'grito', 'procesen', '.', 'por', 'cantar', ',', 'canta', ',', 'cantar', ',', 'cantar', ',', 'cantáis', ',', 'cantar', '…']


La lematización es un proceso clave en muchas tareas prácticas de PLN, pero tiene ***dos costos***:   
- Primero, es un proceso que consume recursos (sobre todo tiempo). 
- Segundo, suele ser probabilística, así que en algunos casos se suelen obtener resultados inesperados.

## STEMMING

Es el procedimiento de convertir palabras en raíces. Estas raíces son la parte invariable de palabras relacionadas sobre todo por su forma.    

En cierta manera se parece a la lematización, pero los resultados (las raíces) no tienen por qué ser palabras de un idioma.  

In [13]:
import nltk
from nltk import SnowballStemmer
spanishstemmer=SnowballStemmer('spanish')
text = """Soy un texto que pide a gritos que lo procesen. Por eso yo canto, tú cantas, ella canta, nosotros cantamos, cantáis, cantan…"""
tokens = normalize(text) # crear una lista de tokens
stems = [spanishstemmer.stem(token) for token in tokens]
print(stems)

['text', 'pid', 'grit', 'proces', 'cant', 'cant', 'cant', 'cant', 'cant', 'cant']


Como se puede observar, para encontrar las raíces en español nos hemos valido de otra librería de Python llamada **nltk**. Es otra librería fundamental para el procesamiento de lenguaje natural.    

En **nltk** hay muchas funciones para tareas de este tipo, en varios idiomas. En el ejemplo se ha utilizado el ***Snowball Stemmer*** porque funciona no sólo en inglés, sino en otras lenguas como el español.

El stemming es mucho más rápido desde el punto de vista del procesamiento que la lematización. También tiene como ventaja que reconoce relaciones entre palabras de distinta clase. Podría reconocer, por ejemplo, que picante y picar tienen como raíz pic-. En otras palabras, el stemming puede reducir el número de elementos que forman nuestros textos. Y eso, en muchos casos, es lo se suele tener como objetivo.

Por otra parte, una desventaja del stemming es que sus algoritmos son más simples que los de lematización. Pueden “recortar” demasiado la raíz y encontrar relaciones entre palabras que realmente no existen (overstemming).    
También puede suceder que deje raíces demasiado extensas o específicas, y que tengamos más bien un déficit de raíces (understemming), en cuyo caso palabras que deberían convertirse en una misma raíz no lo hacen. No hay mucho que hacer con eso, pero el stemming es una muy buena solución de compromiso en la mayoría de los casos.

## Conclusión   

La **tokenización**, **normalización**, **lematización** y **“radicalización”(stemming)** de un texto suelen ser procedimientos fundamentales para muchas tareas relacionadas con la extracción automática de características y de datos de los textos.    
Estos procedimientos están en la base de los grandes y pequeños buscadores de información.    

El **stemming** suele ser una buena solución cuando no importa demasiado la precisión y se requiere de un procesamiento eficiente.    
La **lematización** suele funcionar mejor cuando se necesita procesar palabras de manera similar a como lo hace un ser humano.